In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import cairosvg

# Configuration du navigateur
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

pages = ['https://fr.wiktionary.org/w/index.php?title=Catégorie:Radicaux_en_arabe&pageuntil=ق+ي+م#mw-pages',
         'https://fr.wiktionary.org/w/index.php?title=Catégorie:Radicaux_en_arabe&pagefrom=ق+ي+م#mw-pages']

urls = []

In [5]:
for page in pages:

    driver.get(page)

    # Création des liens
    search_i = True ; i = 1
    while search_i:
        try:
            _ = driver.find_element(By.XPATH, f'//*[@id="mw-pages"]/div/div/div[{i}]/ul/li[1]/a').is_displayed()

            search_j = True ; j = 1
            while search_j:
                try:
                    urls.append(driver.find_element(By.XPATH, f'//*[@id="mw-pages"]/div/div/div[{i}]/ul/li[{j}]/a').get_attribute('href'))
                    j += 1

                except:
                    search_j = False
            
            i += 1
        except:
            search_i = False

In [6]:
data = pd.DataFrame(columns=['url', 'radical', 'notions'])

for url in urls:
    driver.get(url)

    try:
        radical = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/dl/dd/b/bdi').text
        notions = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/dl/dd/ol/li').text
    except:
        try:
            radical = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/dl[2]/dd/b/bdi').text
            notions = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/dl[2]/dd/ol/li').text
        except:
            radical = ''
            notions = ''

    raw = {'url': page, 'radical': radical, 'notions': notions}
    data = pd.concat([data, pd.DataFrame(raw, index=[0])], ignore_index=True)

data = data[(data['radical'] != '') & (data['notions'] != '')]

In [17]:
for index, row in data.iterrows():
    radical = row['radical']
    radical = radical[::-1]
    notions = row['notions']

    svg = f'''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
    <!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
    <svg width="100%" height="100%" viewBox="0 0 2880 1800" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xml:space="preserve" xmlns:serif="http://www.serif.com/" style="fill-rule:evenodd;clip-rule:evenodd;stroke-linejoin:round;stroke-miterlimit:2;">
        <rect x="0" y="0" width="2880" height="1800" style="fill:url(#_Linear1);"/>
        <g transform="matrix(1,0,0,1,-0,89.639)">
            <text x="50%" y="810.361px" text-anchor="middle" style="font-family:'Damascus';font-size:600px;fill:url(#_Linear2);">{radical}</text>
        </g>
        <g transform="matrix(1,0,0,1,-0,930.145)">
            <text x="50%" y="480.484px" text-anchor="middle" style="font-family:'Optima-Regular', 'Optima';font-size:120px;fill:url(#_Linear3);">{notions}</text>
        </g>
        <defs>
            <linearGradient id="_Linear1" x1="0" y1="0" x2="1" y2="0" gradientUnits="userSpaceOnUse" gradientTransform="matrix(2880,0,0,2880,0,1091.57)"><stop offset="0" style="stop-color:rgb(255,144,26);stop-opacity:1"/><stop offset="1" style="stop-color:rgb(255,80,4);stop-opacity:1"/></linearGradient>
            <linearGradient id="_Linear2" x1="0" y1="0" x2="1" y2="0" gradientUnits="userSpaceOnUse" gradientTransform="matrix(2880,0,0,2880,1248.35,630.049)"><stop offset="0" style="stop-color:rgb(255,144,26);stop-opacity:1"/><stop offset="1" style="stop-color:rgb(255,200,67);stop-opacity:1"/></linearGradient>
            <linearGradient id="_Linear3" x1="0" y1="0" x2="1" y2="0" gradientUnits="userSpaceOnUse" gradientTransform="matrix(2880,0,0,2880,1248.35,630.049)"><stop offset="0" style="stop-color:rgb(89,1,39);stop-opacity:1"/><stop offset="1" style="stop-color:rgb(220,62,1);stop-opacity:1"/></linearGradient>
        </defs>
    </svg>
    '''

    # Chemin de sortie pour l'image générée
    chemin_image = f'/Users/gauthier/Desktop/radicaux/images/image{index}.png'

    # Conversion du SVG en image
    cairosvg.svg2png(bytestring=svg, write_to=chemin_image)